In [ ]:
# this cell imports relevant modules and configures later cells

import pandas as pd
import mpl_scatter_density # adds projection='scatter_density'
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, Normalize

# color scheme for the COP analysis scatter plot
white_viridis = LinearSegmentedColormap.from_list('white_viridis', [
    (0, '#ffffff'),
    (1e-20, '#440053'),
    (0.2, '#404388'),
    (0.4, '#2a788e'),
    (0.6, '#21a784'),
    (0.8, '#78d151'),
    (1, '#fde624'),
], N=256)

TIMESTEP_SECONDS = 900
AGG_STEP_SECONDS = 3600
LEAP_YEARS = ['2024']
TIME_ZONE = "Europe/Berlin"
ENERGY_UNIT = "kWh"
AGGREGATION = "1h"
DELIMITER = ";"
DECIMAL = ","
TIMESTAMP_COLUMNS = ["timestamp"]
RENAME_COLUMNS = {}
FILTER_BY_ACTIVE = True
CLIP_CAP_MSR = 10
CLIP_CAP_SIM = 20

In [ ]:
# this cell loads the measurement data and prepares it for further analysis

def add_additional_columns(data):
    """Adds additional columns to the data, that are calculated from the measurement data."""
    # if the measurements have meter values instead of values relative to the
    # time step, these can be calculated as the difference between meter values.
    if 'el_in meter' in data:
        data['el_in'] = data['el_in meter'].diff()
        data.loc[data.index[0], 'el_in'] = 0
    if 'heat_in meter' in data:
        data['heat_in'] = data['heat_in meter'].diff()
        data.loc[data.index[0], 'heat_in'] = 0
    if 'heat_out meter' in data:
        data['heat_out'] = data['heat_out meter'].diff()
        data.loc[data.index[0], 'heat_out'] = 0

    data['temp diff'] = data['heat_out temp'] - data['heat_in temp']
    data['active'] = (data['heat_in'] > 0) & (data['el_in'] > 0)
    data['cop'] = data['heat_out'] / data['el_in']

    # the absolute residual is a measure for how much the energy balance in the measurements
    # differs from the theoretical case of `heat_out = heat_in + el_in`
    if 'heat_in' in data:
        data['abs res'] = abs(
            (data['heat_in'] + data['el_in'] - data['heat_out'])
            / data['heat_out']
        )

# read measurements
file_path = 'measurements_all.csv'
data = pd.read_csv(file_path, delimiter=DELIMITER, decimal=DECIMAL, na_values=["\\N", ""])

if len(TIMESTAMP_COLUMNS) > 1:
    data['timestamp'] = pd.to_datetime(data[TIMESTAMP_COLUMNS])
else:
    data['timestamp'] = pd.to_datetime(data[TIMESTAMP_COLUMNS[0]])
# remove leap days as the simulation does not support them.
data = data[~((data['timestamp'].dt.month == 2) & (data['timestamp'].dt.day == 29))]
# now set the index
data.set_index('timestamp', inplace=True)

# apply corrections to the data
corrections_file_path = 'corrections.csv'
corrections = pd.read_csv(corrections_file_path, delimiter=DELIMITER, decimal=DECIMAL)
corrections['timestamp'] = pd.to_datetime(corrections['timestamp'])

for _, row in corrections.iterrows():
    timestamp = row['timestamp']
    for column in corrections.columns:
        if column != 'timestamp':
            data.loc[timestamp, column] = row[column]

# rename columns, if necessary, and add additional columns
if RENAME_COLUMNS != {}:
    data.rename(inplace=True, errors="raise", columns=RENAME_COLUMNS)
add_additional_columns(data)

# if we have meter values, the removal of leap days leaves the first row after one with a
# big spike in values. we set the values to zero as the leap day should not be counted
# at all
if 'heat_out meter' in data:
    for year in LEAP_YEARS:
        data.loc[f'{year}-03-01 00:00:00', "el_in"] = 0.0
        data.loc[f'{year}-03-01 00:00:00', "heat_in"] = 0.0
        data.loc[f'{year}-03-01 00:00:00', "heat_out"] = 0.0

# aggregate data to a larger timestep
data_agg = data.resample(AGGREGATION).agg({
    'heat_in temp': 'mean',
    'heat_out temp': 'mean',
    'heat_out': 'sum',
    'heat_in': 'sum',
    'el_in': 'sum',
})
add_additional_columns(data_agg)

# write out an extract of the data
with open('extract.txt', 'w') as file:
    file.write(data.head(1000).to_string())


In [ ]:
# this cell create profiles from measurement data for simulation with ReSiE
profiles = [
    ('heat_out', "heat_demand.prf"),
    ('heat_in temp', "heat_source_temp.prf"),
    ('heat_out temp', "heat_demand_temp.prf"),
]
start_date = data.index[0].strftime("%d.%m.%Y %H:%M:%S")

for profile in profiles:
    data_type = "extensive" if profile[0] == "heat_out" else "intensive"
    # rescale energy values as ReSiE works with Wh, but the data might use a different unit
    scale_factor = 1.0
    if profile[0] == "heat_out" and ENERGY_UNIT == "kWh":
        scale_factor = 1000.0  
    elif profile[0] == "heat_out" and ENERGY_UNIT == "MWh":
        scale_factor = 1000000.0

    # profile metadata
    with open(profile[1], 'w') as file:
        file.write(f"# data_type:                 {data_type}\n")
        file.write("# time_definition:           startdate_timestepsize\n")
        file.write(f"# time_zone:                 {TIME_ZONE}\n")
        file.write(f"# profile_start_date:        {start_date}\n")
        file.write("# profile_start_date_format: dd.mm.yyyy HH:MM:SS\n")
        file.write(f"# profile_time_step_seconds: {TIMESTEP_SECONDS}\n")

    # data without timestamp, as the profile uses a start date and a fixed time step instead
    with open(profile[1], 'a') as file:
        data_tmp = pd.DataFrame()
        #data_tmp["timestamp"] = data.index
        data_tmp[profile[0]] = scale_factor * data[profile[0]]
        data_tmp.to_csv(file, index=False, sep=';', decimal='.', header=False, lineterminator='\n')

In [ ]:
# this cell loads the results of the simulation and prepares the data for further analysis

file_path = 'output/resie_out.csv'
data_sim = pd.read_csv(file_path, delimiter=";", decimal=",", na_values=["\\N"])
data_sim['timestamp'] = pd.to_datetime(data_sim['Time [dd.mm.yyyy HH:MM:SS]'], format='%d.%m.%Y %H:%M:%S')
data_sim.set_index('timestamp', inplace=True)

data_sim.rename(errors="raise", inplace=True, columns={
    'TST_HP_01 m_e_ac_230v IN': 'el_in',
    'TST_HP_01 m_h_w_lt1 IN': 'heat_in',
    'TST_HP_01 m_h_w_ht1 OUT': 'heat_out',
    'TST_SRC_01 Temperature': 'heat_in temp',
    'TST_DEM_01 Temperature': 'heat_out temp',
})

# rescale simulation data so it matches the unit of measurements
scale_factor = 1.0
if ENERGY_UNIT == "kWh":
    scale_factor = 1.0 / 1000.0  
elif ENERGY_UNIT == "MWh":
    scale_factor = 1.0 / 1000000.0
data_sim['heat_in'] = data_sim['heat_in'] * scale_factor
data_sim['heat_out'] = data_sim['heat_out'] * scale_factor
data_sim['el_in'] = data_sim['el_in'] * scale_factor

add_additional_columns(data_sim)

data_sim_agg = data_sim.resample(AGGREGATION).agg({
    'el_in': 'sum',
    'heat_in': 'sum',
    'heat_out': 'sum',
    'heat_in temp': 'mean',
    'heat_out temp': 'mean',
})
add_additional_columns(data_sim_agg)

In [ ]:
# this cell performs the COP analysis for the measurements and the simulated data and 
# creates scatter plots that display this analysis

def using_mpl_scatter_density(fig, axis, x, y, max_density):
    """Configures the figure/axis to use scatter density method."""
    density = axis.scatter_density(
        x, y, cmap=white_viridis, norm=Normalize(vmin=0, vmax=max_density, clip=True)
    )
    colorbar = fig.colorbar(density, ax=axis, orientation='vertical')
    colorbar.set_label(f"Number of points per pixel (clipped at {max_density})")

def plot_quadrant(fig, axis, data, x_def, y_def, clip_cap=20, dt=15, is_sim=False):
    """Plots the given data as one quadrant of the scatter plot with the density method."""
    using_mpl_scatter_density(fig, axis, data[x_def["name"]], data[y_def["name"]], clip_cap)
    title = f"{'Simulation' if is_sim else 'Measurement'} data " + \
        f"with {len(data[x_def['name']])} values (Δt={dt} min)\n" + \
        f"of {x_def['title']} vs {y_def['title']}"
    axis.set_title(title)
    axis.set_xlabel(x_def["title"])
    axis.set_ylabel(y_def["title"])
    axis.set_xlim(x_def["lo_lim"], x_def["hi_lim"])
    axis.set_ylim(y_def["lo_lim"], y_def["hi_lim"])

def filter_data(data, x_def, y_def):
    """Filters the given data with given filter criteria.
    
    The filter for column 'active' is only used if the validation is configured via config
    FILTER_BY_ACTIVE that way. The filter for the absolute residual is only used if the
    column exists, as this might not be the case.
    """
    return data[
        (not FILTER_BY_ACTIVE or data['active'])
        & ("abs res" not in data or data['abs res'] < 1.0)
        & (data[x_def["name"]] >= x_def["lo_lim"])
        & (data[x_def["name"]] <= x_def["hi_lim"])
        & (data[y_def["name"]] >= y_def["lo_lim"])
        & (data[y_def["name"]] <= y_def["hi_lim"])
    ]

# scatter plots for delta T vs COP with density as color. the four quadrants cover the
# variations of original vs aggregated time step as well as measurements vs simulation
fig, axes = plt.subplots(2, 2, figsize=(16, 12), subplot_kw={'projection': "scatter_density"})
x_def = {"name": "temp diff", "title": "Temperature difference [K]", "lo_lim": 15, "hi_lim": 60}
y_def = {"name": "cop", "title": "COP [-]", "lo_lim": 1, "hi_lim": 6}

axis = axes[0,0]
filtered = filter_data(data, x_def, y_def)
plot_quadrant(fig, axis, filtered, x_def, y_def, CLIP_CAP_MSR, int(TIMESTEP_SECONDS / 60), False)

axis = axes[0,1]
filtered = filter_data(data_sim, x_def, y_def)
plot_quadrant(fig, axis, filtered, x_def, y_def, CLIP_CAP_SIM, int(TIMESTEP_SECONDS / 60), True)

axis = axes[1,0]
filtered = filter_data(data_agg, x_def, y_def)
plot_quadrant(fig, axis, filtered, x_def, y_def, CLIP_CAP_MSR, int(AGG_STEP_SECONDS / 60), False)

axis = axes[1,1]
filtered = filter_data(data_sim_agg, x_def, y_def)
plot_quadrant(fig, axis, filtered, x_def, y_def, CLIP_CAP_SIM, int(AGG_STEP_SECONDS / 60), True)

plt.tight_layout()
plt.savefig("cop_analysis.png")
plt.show()

In [ ]:
# this cell shows energies and COP of measurements vs simulations as line plots over the
# timespan of the data

# resample as daily values and remove leap days (as the simulation does not support them)
filtered = data[~((data.index.month == 2) & (data.index.day == 29))]
data_daily = filtered.resample('1d').agg({
    'heat_out': 'sum',
    'heat_in': 'sum',
    'el_in': 'sum',
})
data_daily['cop'] = data_daily['heat_out'] / data_daily['el_in']

data_sim_daily = data_sim.resample('1d').agg({
    'heat_out': 'sum',
    'heat_in': 'sum',
    'el_in': 'sum',
})
data_sim_daily['cop'] = data_sim_daily['heat_out'] / data_sim_daily['el_in']

# line plots
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# plot for COP
axis_cop = axes[0]
axis_nrg = axis_cop.twinx() # Create a twin y-axis for heat demand
data_daily['heat_out'].plot(ax=axis_nrg, label='Heat demand [kWh]', color='green', linestyle='--', alpha=0.5)
data_daily['cop'].plot(ax=axis_cop, label='Measured COP [-]', color='blue')
data_sim_daily['cop'].plot(ax=axis_cop, label='Simulated COP [-]', color='orange')
axis_nrg.set_title("Measured vs simulated COP")
axis_nrg.set_xlabel("Date [-]")
axis_nrg.set_ylabel("Energy [kWh]")
axis_nrg.set_xlim(data_daily.index[0], data_daily.index[-1])
axis_nrg.set_ylim(0, 2500)
axis_nrg.legend(loc='upper right')
axis_cop.set_ylim(1, 5)
axis_cop.set_ylabel("COP [-]")
axis_cop.legend(loc='upper left')
axis_cop.grid()

# plot for heat input
axis_nrg = axes[1]
axis_cop = axis_nrg.twinx() # Create a twin y-axis for COP
data_daily['cop'].plot(ax=axis_cop, label='Measured COP [-]', color='green', linestyle='--', alpha=0.5)
data_daily['heat_in'].plot(ax=axis_nrg, label='Measured heat input [kWh]', color='blue')
data_sim_daily['heat_in'].plot(ax=axis_nrg, label='Simulated heat input [kWh]', color='orange')
axis_nrg.set_title("Measured vs simulated heat input")
axis_nrg.set_xlabel("Date [-]")
axis_nrg.set_ylabel("Energy [kWh]")
axis_nrg.set_xlim(data_daily.index[0], data_daily.index[-1])
axis_nrg.set_ylim(0, 1500)
axis_nrg.legend(loc='upper left')
axis_cop.set_ylim(1, 5)
axis_cop.set_ylabel("COP [-]")
axis_cop.legend(loc='upper right')
axis_nrg.grid()

# plot for electricity input
axis_nrg = axes[2]
axis_cop = axis_nrg.twinx() # Create a twin y-axis for COP
data_daily['cop'].plot(ax=axis_cop, label='Measured COP [-]', color='green', linestyle='--', alpha=0.5)
data_daily['el_in'].plot(ax=axis_nrg, label='Measured electricity input [kWh]', color='blue')
data_sim_daily['el_in'].plot(ax=axis_nrg, label='Simulated electricity input [kWh]', color='orange')
axis_nrg.set_title("Measured vs simulated electricity input")
axis_nrg.set_xlabel("Date [-]")
axis_nrg.set_ylabel("Energy [kWh]")
axis_nrg.set_xlim(data_daily.index[0], data_daily.index[-1])
axis_nrg.set_ylim(0, 1500)
axis_nrg.legend(loc='upper left')
axis_cop.set_ylim(1, 5)
axis_cop.set_ylabel("COP [-]")
axis_cop.legend(loc='upper right')
axis_nrg.grid()

plt.tight_layout()
plt.savefig("daily_data_full.png")
plt.show()

In [ ]:
# this cell creates line plots comparing measured vs simulated data on a finer-grained
# scale than the previous line plots

# line plots
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# plot for COP
axis_cop = axes[0]
axis_nrg = axis_cop.twinx() # Create a twin y-axis for heat demand
data_agg['heat_out'].plot(ax=axis_nrg, label='Heat demand [kWh]', color='green', linestyle='--', alpha=0.5)
data_agg['cop'].plot(ax=axis_cop, label='Measured COP [-]', color='blue')
data_sim_agg['cop'].plot(ax=axis_cop, label='Simulated COP [-]', color='orange')
axis_nrg.set_title("Measured vs simulated COP")
axis_nrg.set_xlabel("Date [-]")
axis_nrg.set_ylabel("Energy [kWh]")
axis_nrg.set_xlim(data_agg.index[8556], data_agg.index[9300])
axis_nrg.set_ylim(0, 120)
axis_nrg.legend(loc='upper right')
axis_cop.set_ylim(1, 5)
axis_cop.set_ylabel("COP [-]")
axis_cop.legend(loc='upper left')
axis_cop.grid()


# plot for heat input
axis_nrg = axes[1]
axis_cop = axis_nrg.twinx() # Create a twin y-axis for COP
data_agg['cop'].plot(ax=axis_cop, label='Measured COP [-]', color='green', linestyle='--', alpha=0.5)
data_agg['heat_in'].plot(ax=axis_nrg, label='Measured heat input [kWh]', color='blue')
data_sim_agg['heat_in'].plot(ax=axis_nrg, label='Simulated heat input [kWh]', color='orange')
axis_nrg.set_title("Measured vs simulated heat input")
axis_nrg.set_xlabel("Date [-]")
axis_nrg.set_ylabel("Energy [kWh]")
axis_nrg.set_xlim(data_agg.index[8556], data_agg.index[9300])
axis_nrg.set_ylim(0, 120)
axis_nrg.legend(loc='upper left')
axis_cop.set_ylim(1, 5)
axis_cop.set_ylabel("COP [-]")
axis_cop.legend(loc='upper right')
axis_nrg.grid()

# plot for electricity input
axis_nrg = axes[2]
axis_cop = axis_nrg.twinx() # Create a twin y-axis for COP
data_agg['cop'].plot(ax=axis_cop, label='Measured COP [-]', color='green', linestyle='--', alpha=0.5)
data_agg['el_in'].plot(ax=axis_nrg, label='Measured electricity input [kWh]', color='blue')
data_sim_agg['el_in'].plot(ax=axis_nrg, label='Simulated electricity input [kWh]', color='orange')
axis_nrg.set_title("Measured vs simulated electricity input")
axis_nrg.set_xlabel("Date [-]")
axis_nrg.set_ylabel("Energy [kWh]")
axis_nrg.set_xlim(data_agg.index[8556], data_agg.index[9300])
axis_nrg.set_ylim(0, 120)
axis_nrg.legend(loc='upper left')
axis_cop.set_ylim(1, 5)
axis_cop.set_ylabel("COP [-]")
axis_cop.legend(loc='upper right')
axis_nrg.grid()

plt.tight_layout()
plt.savefig("aggregated_one_month.png")
plt.show()